In [ ]:
!pip install openai langchain langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00


In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.espn.com/")

# Day2- RAG로 캠페인의 광고 도달 분석하기

In [ ]:
from openai import OpenAI

def query_open_api(input_text, system_q='You are a helpful assistant.'):
  client = OpenAI()
  completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": system_q},
        {
            "role": "user",
            "content": input_text
        }
    ]
  )
  return completion.choices[0].message.content


In [ ]:
import pandas as pd

In [ ]:
# prompt: pandas dataframe을 openapi prompt 데이터로 변환하는 코드 생성해줘

def df_to_openapi_prompt(df, desc_data, query):
  """Converts a Pandas DataFrame into an OpenAPI prompt suitable for GPT-4.

  Args:
    df: The Pandas DataFrame to convert.

  Returns:
    A string representing the OpenAPI prompt.
  """

  table_data = df.to_string()
  prompt = f"""
  You are a helpful assistant that can answer questions based on provided data.
  {desc_data}

  **Table Data:**
  {table_data}

  Query: {query}
  """

  return prompt


def data_to_openapi_prompt(data_string, desc_data, query):
  """Converts a Pandas DataFrame into an OpenAPI prompt suitable for GPT-4.

  Args:
    df: The Pandas DataFrame to convert.

  Returns:
    A string representing the OpenAPI prompt.
  """

  prompt = f"""
  You are a helpful assistant that can answer questions based on provided data.
  {desc_data}

  ** Data:**
  {data_string}

  Query: {query}
  """

  return prompt

In [ ]:
reach_ds = pd.read_csv('medicube_user_reach_90d_20250310_20250310_name.tsv')
# prompt: reach_ds DataFrame 사용:  컬럼명을 읽기 쉬운 것으로 생성해줘

# 컬럼명 변경
new_column_names = {
    'Unnamed: 0': 'index',
    'type_bit': 'type_code',
    'users': 'user_count',
    'trigger_impressions': 'trigger_impression_count',
    'impressions': 'impression_count',
    'clicks': 'click_count',
    'conversions': 'conversion_count',
    '광고활동': 'campaign_access_info'
}

reach_ds.rename(columns=new_column_names, inplace=True)
reach_ds.campaign_access_info = reach_ds.campaign_access_info.apply(lambda x: x.replace('신규', '캠페인'))

In [ ]:
reach_ds.groupby('only_campaign').agg({'user_count':sum}).sort_values('user_count', ascending=False)

<ipython-input-257-70c53ebe268f>:1: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  reach_ds.groupby('only_campaign').agg({'user_count':sum}).sort_values('user_count', ascending=False)


,user_count
only_campaign,
이전광고노출,9431718
캠페인도달 AND 이전광고노출,9259753
캠페인도달,5656618
,2515276
캠페인도달 AND 광고클릭 AND 이전광고노출,78129
캠페인도달 AND 광고클릭,28667
캠페인도달 AND 광고전환 AND 이전광고노출,507
광고클릭 AND 이전광고노출,347
캠페인도달 AND 광고클릭 AND 광고전환 AND 이전광고노출,326


In [ ]:
q = (df_to_openapi_prompt(reach_ds[['campaign_access_info', 'user_count']],
                          "메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.",
                          "캠페인이 광고 도달의 기여 효과를 설명해줘"))
print(q)


  You are a helpful assistant that can answer questions based on provided data.
  메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.

  **Table Data:**
                                  campaign_access_info  user_count
0                          캠페인도달 AND 광고전환 AND 이전광고노출         182
1            캠페인트리거뷰도달 AND 캠페인도달 AND 광고전환 AND 이전광고노출         325
2                                               광고클릭         212
3                            캠페인도달 AND 광고클릭 AND 광고전환          10
4                                             이전광고노출     6827345
5            캠페인트리거뷰도달 AND 캠페인도달 AND 광고클릭 AND 이전광고노출       57288
6                     캠페인트리거뷰도달 AND 캠페인도달 AND 이전광고노출     5867708
7                                          캠페인트리거뷰도달     2515276
8   캠페인트리거뷰도달 AND 캠페인도달 AND 광고클릭 AND 광고전환 AND 이전광고노출         265
9                      캠페인트리거뷰도달 AND 광고전환 AND 이전광고노출         103
10                                캠페인트리거뷰도달 AND 광고클릭          60
11                              캠페인트리거뷰도달 AND 이전광고노출     2604373
12   

In [ ]:
def show_me_insight_from_table(table_ds, table_desc, question
                               , system_q="you are expert data analyst in ad tech domain"):
  q = (df_to_openapi_prompt(table_ds,
                          table_desc,
                          question))
  print(f'''Prompt:
  {q}
  -------------------------------------------------------
  ''')
  result = query_open_api(q, system_q=system_q)
  print(result)

  return result


def show_me_insight_text(text, table_desc, question, system_q="you are expert data analyst in ad tech domain"):
  q = (data_to_openapi_prompt(text,
                          table_desc,
                          question))
  print(f'''Prompt:
  {q}
  -------------------------------------------------------
  ''')
  result = query_open_api(q, system_q)
  print(result)

  return result




In [ ]:
reach_ds['only_campaign'] = reach_ds.campaign_access_info.apply(lambda x: x.replace('캠페인트리거뷰도달 AND', '').replace('캠페인트리거뷰도달', '').strip())

In [ ]:
reach_ds[(reach_ds.type_code & 2)>0].sum()

,0
index,163
type_code,227
user_count,15024036
trigger_impression_count,36603357
impression_count,62344387
click_count,118651
conversion_count,939
campaign_access_info,캠페인도달 AND 광고전환 AND 이전광고노출캠페인트리거뷰도달 AND 캠페인도달 A...
only_campaign,캠페인도달 AND 광고전환 AND 이전광고노출 캠페인도달 AND 광고전환 AND 이...


In [ ]:
r = show_me_insight_from_table(reach_ds[['campaign_access_info', 'user_count']],
                "메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.",
                """테이블의 campaign_access_info에서 캠페인트리거뷰도달은 없애고 campaign_access_info 다시 그룹합산하여 결과물을 CSV 형식으로 표현해줘
                아래처럼 캠페인트리거뷰도달을 제거하면 하나의 그룹으로 합칩니다.
                 - 캠페인트리거뷰도달 AND 캠페인도달 : 캠페인도달
                 - 캠페인도달 : 캠페인도달
                 - 캠페인트리거뷰도달 AND 캠페인도달 AND 이전광고노출 : 캠페인도달 AND 이전광고노출

                SYSTEM: 결과CSV만 출력해줘. 정렬도 해줘
              """)


Prompt:
  
  You are a helpful assistant that can answer questions based on provided data.
  메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.

  **Table Data:**
                                  campaign_access_info  user_count
0                          캠페인도달 AND 광고전환 AND 이전광고노출         182
1            캠페인트리거뷰도달 AND 캠페인도달 AND 광고전환 AND 이전광고노출         325
2                                               광고클릭         212
3                            캠페인도달 AND 광고클릭 AND 광고전환          10
4                                             이전광고노출     6827345
5            캠페인트리거뷰도달 AND 캠페인도달 AND 광고클릭 AND 이전광고노출       57288
6                     캠페인트리거뷰도달 AND 캠페인도달 AND 이전광고노출     5867708
7                                          캠페인트리거뷰도달     2515276
8   캠페인트리거뷰도달 AND 캠페인도달 AND 광고클릭 AND 광고전환 AND 이전광고노출         265
9                      캠페인트리거뷰도달 AND 광고전환 AND 이전광고노출         103
10                                캠페인트리거뷰도달 AND 광고클릭          60
11                              캠페인트리거뷰도달 AND 이전광고노출     260

In [ ]:
r2 = show_me_insight_text(r
                     , """메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.
                     결과는 간결하게 개조식으로 서술해줘"""
                     , """캠페인의 광고 도달 관점에서 기여를 정량적으로 측정하고자 합니다.
                     - 전체 캠페인 도달수 중 이전광고노출 안된 도달수와 그 비중
                     - 전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 클릭 기여 분석
                     - 전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 전환 기여 분석
                     """
                     , "너는 데이터분석 전문가이고 광고도메인전문가이다.")

Prompt:
  
  You are a helpful assistant that can answer questions based on provided data.
  메디큐브의 보장형 광고 집행 데이터이고, 이전 광고가 도달까지 포함 된 정보이다.
                     결과는 간결하게 개조식으로 서술해줘

  ** Data:**
  ```csv
campaign_access_info,user_count
광고전환 AND 이전광고노출,118
광고클릭,212
광고클릭 AND 이전광고노출,184
캠페인도달,7902896
캠페인도달 AND 광고전환 AND 이전광고노출,182
캠페인도달 AND 광고클릭,28667
캠페인도달 AND 광고클릭 AND 광고전환,36
캠페인도달 AND 광고클릭 AND 광고전환 AND 이전광고노출,61
캠페인도달 AND 광고클릭 AND 이전광고노출,78018
캠페인도달 AND 이전광고노출,9259753
이전광고노출,6827345
```

  Query: 캠페인의 광고 도달 관점에서 기여를 정량적으로 측정하고자 합니다.
                     - 전체 캠페인 도달수 중 이전광고노출 안된 도달수와 그 비중
                     - 전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 클릭 기여 분석
                     - 전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 전환 기여 분석
                     
  
  -------------------------------------------------------
  
- **전체 캠페인 도달수 중 이전광고노출 안된 도달수와 그 비중:**
  - 전체 캠페인 도달수: 7,902,896
  - 이전광고노출 없는 도달수 계산:  
    - 전체 캠페인 도달수 - 캠페인도달 AND 이전광고노출 수  
    - = 7,902,896 - 9,259,753 (이는 음수가 되므로 이미 이전광고노출된 모든 수를 포함한다고 볼 

In [ ]:
# prompt: markdown text를 jupyter notebook markdown 디스플레이하는 코드

from IPython.display import Markdown

Markdown(r2)

- **전체 캠페인 도달수 중 이전광고노출 안된 도달수와 그 비중:**
  - 전체 캠페인 도달수: 7,902,896
  - 이전광고노출 없는 도달수 계산:  
    - 전체 캠페인 도달수 - 캠페인도달 AND 이전광고노출 수  
    - = 7,902,896 - 9,259,753 (이는 음수가 되므로 이미 이전광고노출된 모든 수를 포함한다고 볼 수 있음)
  - 이전광고노출 안된 비중은 이 데이터에서는 직접적으로 해석 불가

- **전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 클릭 기여 분석:**
  - 캠페인도달 AND 광고클릭: 28,667
  - 캠페인도달 AND 광고클릭 AND 이전광고노출: 78,018
  - 캠페인도달 중 광고클릭 기여도 계산:
    - (캠페인도달 AND 광고클릭) / 캠페인도달 * 100
    - = (28,667 / 7,902,896) * 100 = 약 0.36%
  - 이전광고노출 없는 클릭 기여도 비교적 불명확하지만, 노출된 클릭 수가 높음을 인식

- **전체 캠페인에서 이전광고 도달과 도달되지 않은 광고의 전환 기여 분석:**
  - 캠페인도달 AND 광고전환: 불명시 - 간접 계산 필요
  - 캠페인도달 AND 광고전환 AND 이전광고노출: 182
  - 광고전환 수 계산 (전환 측면에서 해석):  
    - = 캠페인도달 AND 광고전환 AND 이전광고노출 + 캠페인도달 AND 광고클릭 AND 광고전환 AND 이전광고노출
    - = 182 + 61 = 243  
  - 캠페인도달 중 전환 기여도 계산:
    - (캠페인도달 AND 광고전환 수) / 캠페인도달 * 100
    - 약 0.0031%
  - 이전광고노출 있는 상태에서의 전환 기여가 대부분이며, 일반 전환보다 높음을 확인  

위의 데이터로는 이전광고 노출 없는 기여 상황을 정밀하게 판단하기 충분하지 않으므로 추가적인 데이터나 분석 방법이 필요합니다.

[]

# Day2 프로젝트

In [ ]:
%%capture --no-stderr
%pip install langchain langchain-openai langchain-openai langchain_chroma langchain-text-splitters langchain_community

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

··········


In [ ]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

example_messages

[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: filler question \nContext: filler context \nAnswer:", additional_kwargs={}, response_metadata={})]

In [ ]:
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: filler question 
Context: filler context 
Answer:


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter


urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model='text-embedding-3-small'))


In [ ]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=OpenAIEmbeddings(model='text-embedding-3-small'))

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, more manageable steps. It can be achieved through various methods, including prompting large language models (LLMs) or using task-specific instructions. This technique is often employed to enhance performance on difficult tasks by allowing for a structured approach to reasoning.'

In [ ]:
# etriever search_type 은 'similarity', search_kwargs={'k': 6} 을 사용해주세요.

prompt = PromptTemplate(
    input_variables=["question", "document"],
    template=""" SYSTEM : 너는 문서와 사용자 질문에 대한 연관성을 측정하는 평가자 입니다. 질문의 단어가 문서에 포함되어 있다면 좀더 연관성이 높게 측정합니다.
    맥락과 비슷한 단어이면 연관성이 있다고 판단합니다. the goal is fillter out errorneous retrivals. 결과는 Yes or No 로 json score로 출력합니다.
    Here is retrived document: {document}
    Here is the question: {question}
    """
    )

retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 6})
retrieval_grader = prompt | llm | JsonOutputParser()
question = 'agent memory'
docs = retriever.invoke(question)
doc_text = docs[0].page_content


In [ ]:
print(question)
is_relevant = retrieval_grader.invoke({'question': question,
                                       'document': doc_text})

agent memory


In [ ]:
is_relevant['score']

'Yes'

In [ ]:
prompt_generator = PromptTemplate(
    input_variables=["question", "document"],
    template=""" SYSTEM : 질문 답변을 문제를 풀자.
    Here is retrived document: {document}
    Here is the question: {question}
    """
    )

reg_generation = prompt_generator | llm | JsonOutputParser()
